In [2]:
from tagger.models.received_item import ReceivedItem, ReceivedItemStatus
from tagger.core.mongo.models.order import Order, OrderStatus
from tagger.viewsets.order.change_status import ChangeStatusSerializer, _change_status
from django.contrib.auth.models import User
from tagger.models.inventory import Inventory, InventoryStatus
from tagger.models import ShippingNotice, Package, Courier
from tagger.models.package import PackageStatus

import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
import pandas as pd

courier = Courier.objects.get(name="CJ대한통운")

In [4]:
with open("../Downloads/확장주문검색_20211117_154740.xlsx", "rb") as f:
    df = pd.read_excel(f, dtype=str)
    df.dropna(subset=["주문번호"], inplace=True)
    for _, row in df.iterrows():
        print("--------------------------------------------------------------------------------")
        package_code = row["주문번호"]
        package = Package.objects.get(code=package_code)
        print(row["주문번호"], row["판매처 상품명"], row["수령자이름"])
        for item in package.shipping_notice_items.all():
            print(f"""ㄴ ORDER {item.item.extended_order.code} ({item.item.extended_order.order.payment.buyername}) : ITEM {item.item.product_code}""")

--------------------------------------------------------------------------------
PKG-211117-QX 벨티드 후드 푸퍼 다운점퍼(카키) 한윤경
ㄴ ORDER 4G3DC (한윤경) : ITEM BEANPOLEL___벨티드후드푸퍼다운점퍼(카키)___90
--------------------------------------------------------------------------------
PKG-211117-4J [빈폴스포츠] (MEN) 포켓 구스 점퍼(아이보리) 허윤정
ㄴ ORDER FDVPK (허윤정) : ITEM BEANPOLEL___[빈폴스포츠](MEN)포켓구스점퍼(아이보리)___100
--------------------------------------------------------------------------------
PKG-211117-4U [빈폴스포츠] (MEN) 듀얼 구스 다운 점퍼(네이비) 강영순
ㄴ ORDER AYQDL (강영순 ) : ITEM BEANPOLEL___[빈폴스포츠](MEN)듀얼구스다운점퍼(네이비)___100
--------------------------------------------------------------------------------
PKG-211117-6M [빈폴스포츠] 리브 퀄팅 구스 다운 점퍼(블랙) 구춘화
ㄴ ORDER QYKDV (구춘화) : ITEM BEANPOLEL___[빈폴스포츠]리브퀄팅구스다운점퍼(블랙)___95
--------------------------------------------------------------------------------
PKG-211117-6J [빈폴스포츠] (MEN) 올라운더 보아 구스 롱 다운 권지연
ㄴ ORDER AWCN6 (권지연) : ITEM BEANPOLEL___[빈폴스포츠](MEN)올라운더보아구스롱다운___95
---------------------------------

In [21]:
for _, row in df.iterrows():
    package = Package.objects.get(code=row["주문번호"])
    package.courier = courier
    package.tracking_number = row['송장번호']
    package.status = PackageStatus.SHIPPED
    package.save()
    
    for eo in package.extended_orders:
        order = eo.order
        status_change_data = {
            "status": OrderStatus.DELIVERY_STARTED,
            "delivery_tracking_number": package.courier.name + " " + package.tracking_number,
            "delivery_tracking_url": package.tracking_url,
        }
        s = ChangeStatusSerializer(data=status_change_data, instance=order)
        s.is_valid(raise_exception=True)
        _change_status(s, order, User.objects.get(username="backoffice_server"), id=str(order.id))
        print("-----------------------------------------------------------------------")
        print(f"""[{package.code}] {package.recipient_name} ({package.recipient_mobile})""")
        for item in package.shipping_notice_items.all():
            print(f"ㄴ https://office.alloff.co/orders/{item.item.extended_order.order.code}")
            print(f"""   [INV {item.inventory.code}] {item.item.name}""")
print("-----------------------------------------------------------------------")        

/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField Package.shipped_at received a naive datetime (2021-11-17 07:10:36.981281) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


-----------------------------------------------------------------------
[PKG-211117-QX] 한윤경 (01082230130)
ㄴ https://office.alloff.co/orders/4G3DC
   [INV 211117-017] 벨티드 후드 푸퍼 다운점퍼(카키)


/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField Package.shipped_at received a naive datetime (2021-11-17 07:10:37.377027) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


-----------------------------------------------------------------------
[PKG-211117-4J] 허윤정 (01042900019)
ㄴ https://office.alloff.co/orders/FDVPK
   [INV 211117-014] [빈폴스포츠] (MEN) 포켓 구스 점퍼(아이보리)  
-----------------------------------------------------------------------
[PKG-211117-4U] 강영순  (01026904085)


/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField Package.shipped_at received a naive datetime (2021-11-17 07:10:37.593108) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField Package.shipped_at received a naive datetime (2021-11-17 07:10:37.784442) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


...

/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField Package.shipped_at received a naive datetime (2021-11-17 07:10:40.227695) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField Package.shipped_at received a naive datetime (2021-11-17 07:10:40.420687) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


ㄴ https://office.alloff.co/orders/QCPKK
   [INV 211117-019] 클래식 롱 다운 코트 
-----------------------------------------------------------------------
[PKG-211117-6V] 장정임 (01033025953)
ㄴ https://office.alloff.co/orders/X7XYW
   [INV 211117-008] [빈폴스포츠] (MEN) 올라운더 보아 구스 롱 다운
-----------------------------------------------------------------------
[PKG-211117-6T] 정지영 (01036209940)
ㄴ https://office.alloff.co/orders/KYQTB


/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField Package.shipped_at received a naive datetime (2021-11-17 07:10:40.618758) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


   [INV 211117-002-R] [MaxMara] WSDALIA
-----------------------------------------------------------------------
[PKG-211117-6B] 한신경 (01055446146)
ㄴ https://office.alloff.co/orders/MXNGV
   [INV 211117-013] [빈폴스포츠] (MEN) 포켓 구스 점퍼(아이보리)  
-----------------------------------------------------------------------


/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField Package.shipped_at received a naive datetime (2021-11-17 07:10:40.821637) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


In [10]:
for _, row in df.iterrows():
    codes = split_codes(row["주문번호"])
    for code in codes:
        order = Order.get_by_code(code)
        if order is not None:
            inventories = Inventory.objects.filter(status=InventoryStatus.SHIPPING_PENDING, out_order_id=str(order.id)).all()
            for i in inventories:
                print(f"""{order.code} ({order.id}) - {i.product_name} has been shipped!""")
                i.status = InventoryStatus.SHIPPED
                i.save()

CANMD (618d019d26685ff051756218) - 테일러드 더블 코트 has been shipped!
YR83V (618882f226685ff051755a6c) -  체크패턴 벨티드 코트(멜란지 그레이) has been shipped!
LFRTQ (618dce1826685ff05175634a) - 밍크 퍼 배색 코트  has been shipped!
NARRW (6187f34826685ff051755a02) - 벨티드 플랩 코트(카멜)  has been shipped!
YK4FT (617bed3017fc2e5a1f9eae91) - [로렌 랄프로렌] 서플리스 데님 미디 드레스 has been shipped!
3NYGX (617bcaf817fc2e5a1f9eade4) - [MEN] (7 colors) 울 캐시미어 스웨터 has been shipped!


In [7]:
for i in Inventory.objects.filter(out_order_id__in=[
    "618bdd3d26685ff05175601e",
    "617a9edb17fc2e5a1f9eabe7",
    "618b6a9f26685ff051755f2e",
    "61862bf826685ff0517556c7",
    "6183b47226685ff0517551b1",
    "618a4ca126685ff051755d58",
    "6180d93017fc2e5a1f9eb4ee",
    "618659d826685ff051755731",
    "618265fc26685ff051754f33",
    "6180619517fc2e5a1f9eb42f",
    "6182698926685ff051754f47",
]):
    order = Order.get(i.out_order_id)
    print(f"""{order.code} ({order.id}) - {i.product_name} has been shipped!""")
    i.status = InventoryStatus.SHIPPED
    i.save()

ORD-WAHUH (618bdd3d26685ff05175601e) - 캐시미어 니트 롱 드레스 has been shipped!
ORD-LWH3Y (617a9edb17fc2e5a1f9eabe7) - [택 제거 상품] Teca has been shipped!
ORD-KRJUB (6180619517fc2e5a1f9eb42f) - Torto sweatshirt(네이비) has been shipped!
ORD-X8K9D (6180d93017fc2e5a1f9eb4ee) - [로렌 랄프로렌] 브로드클로스 블루종 슬리브 셔츠 has been shipped!
ORD-PAKK6 (6183b47226685ff0517551b1) - 네오플랜 블록 구스다운 점퍼(DARK GRAY)  has been shipped!
ORD-GKYKH (61862bf826685ff0517556c7) - Ashurst quilted jacket(블랙) has been shipped!
ORD-GL7NK (618265fc26685ff051754f33) - 핸드메이드 캐시미어 코트  has been shipped!
ORD-VERXH (6182698926685ff051754f47) - 핸드메이드 캐시미어 코트  has been shipped!
ORD-BGH46 (618659d826685ff051755731) - [빈폴스포츠] 투웨이 점퍼&조끼 다운 베스트(화이트)  has been shipped!
ORD-9YUM9 (618a4ca126685ff051755d58) - 벨티드 구스 패딩(화이트 핑크)  has been shipped!
ORD-8XRFR (618b6a9f26685ff051755f2e) - 프레이드 트위드 드레스(다크네이비) has been shipped!


True